# Circulation Data Conversion

This document, similar to WIMConversion, is for converting the traffic circulation data into a useable pickled form

In [1]:
import pandas as pd
import numpy as np
import h5py

In [8]:
def load_table_from_struct_circ(table_structure, file) -> pd.DataFrame():

    # get prepared data structure
    data = file[table_structure['table']['data'][0, 1]].value
    # get prepared column names
    data_cols = [name[0] for name in file[table_structure['columns'][0, 0]].value.tobytes()[::2].decode()]

    # create dict out of original table
    table_dict = {}
    for colidx in range(len(data_cols)):
        table_dict[data_cols[colidx]] = [val[0] for val in data[0, 0][0, colidx]]
    
    return pd.DataFrame(table_dict)

In [39]:
station = '137'
year = 2017
file =  h5py.File('{}/{}_{}_struct.mat'.format(station,station, year), 'r')
circ_struct = file['circ_struct']
#df = load_table_from_struct_circ(circ_struct, file)
    # get prepared data structure
data = file[circ_struct['table']['data'][1, 0]].value
data_cols = [name[0] for name in file[circ_struct['columns'][0, :]].value.array2string().tobytes()[::2].decode()]
    

AttributeError: 'numpy.ndarray' object has no attribute 'encode'

In [23]:
def callback(name, obj):
    if name.endswith('type'):
        print('type:', obj.value)
    elif name.endswith('value'):
        if type(obj).__name__=='Dataset':
            print(obj.value.T)  # http://stackoverflow.com/questions/21624653
    elif name.endswith('dims'):
        print('dims:', obj.value)
    else:
        print('name:', name)


In [29]:
file.visititems(callback)

name: #refs#
name: #refs#/0
name: #refs#/0b
name: #refs#/0c
name: #refs#/0d
name: #refs#/0e
name: #refs#/0f
name: #refs#/0g
name: #refs#/0h
name: #refs#/0i
name: #refs#/0j
name: #refs#/0k
name: #refs#/0l
name: #refs#/0m
name: #refs#/0n
name: #refs#/0o
name: #refs#/0p
name: #refs#/0q
name: #refs#/1
name: #refs#/1b
name: #refs#/1c
name: #refs#/1d
name: #refs#/1e
name: #refs#/1f
name: #refs#/1g
name: #refs#/1h
name: #refs#/1i
name: #refs#/1j
name: #refs#/1k
name: #refs#/1l
name: #refs#/1m
name: #refs#/1n
name: #refs#/1o
name: #refs#/1p
name: #refs#/1q
name: #refs#/2
name: #refs#/2b
name: #refs#/2c
name: #refs#/2d
name: #refs#/2e
name: #refs#/2f
name: #refs#/2g
name: #refs#/2h
name: #refs#/2i
name: #refs#/2j
name: #refs#/2k
name: #refs#/2l
name: #refs#/2m
name: #refs#/2n
name: #refs#/2o
name: #refs#/2p
name: #refs#/2q
name: #refs#/3
name: #refs#/3b
name: #refs#/3c
name: #refs#/3d
name: #refs#/3e
name: #refs#/3f
name: #refs#/3g
name: #refs#/3h
name: #refs#/3i
name: #refs#/3j
name: #refs#/3k

name: #refs#/od
name: #refs#/oe
name: #refs#/of
name: #refs#/og
name: #refs#/oh
name: #refs#/oi
name: #refs#/oj
name: #refs#/ok
name: #refs#/ol
name: #refs#/om
name: #refs#/on
name: #refs#/oo
name: #refs#/op
name: #refs#/oq
name: #refs#/or
name: #refs#/p
name: #refs#/pb
name: #refs#/pc
name: #refs#/pd
name: #refs#/pe
name: #refs#/pf
name: #refs#/pg
name: #refs#/ph
name: #refs#/pi
name: #refs#/pj
name: #refs#/pk
name: #refs#/pl
name: #refs#/pm
name: #refs#/pn
name: #refs#/po
name: #refs#/pp
name: #refs#/pq
name: #refs#/pr
name: #refs#/q
name: #refs#/qb
name: #refs#/qc
name: #refs#/qd
name: #refs#/qe
name: #refs#/qf
name: #refs#/qg
name: #refs#/qh
name: #refs#/qi
name: #refs#/qj
name: #refs#/qk
name: #refs#/ql
name: #refs#/qm
name: #refs#/qn
name: #refs#/qo
name: #refs#/qp
name: #refs#/qq
name: #refs#/qr
name: #refs#/r
name: #refs#/rb
name: #refs#/rc
name: #refs#/rd
name: #refs#/re
name: #refs#/rf
name: #refs#/rg
name: #refs#/rh
name: #refs#/ri
name: #refs#/rj
name: #refs#/rk
name: #refs

In [35]:
data

array([[ 10117.,  10117.,  10117., ..., 311217., 311217., 311217.]])

In [37]:
data_cols

['d']

In [51]:
data_dict['circ_struct']

array([([([[array([['Description']], dtype='<U11')], [array([['UserData']], dtype='<U8')], [array([['DimensionNames']], dtype='<U14')], [array([['VariableNames']], dtype='<U13')], [array([['VariableDescriptions']], dtype='<U20')], [array([['VariableUnits']], dtype='<U13')], [array([['VariableContinuity']], dtype='<U18')], [array([['RowNames']], dtype='<U8')], [array([['CustomProperties']], dtype='<U16')]], [[array([['Row']], dtype='<U3'), array([['Variables']], dtype='<U9')]], [[False]], [[3707764736,          2,          1,          1,          1,          1]], [[array([[37093.],
       [37094.],
       [37095.],
       ...,
       [55753.],
       [55754.],
       [55755.]]), array([[ 10117.],
       [ 10117.],
       [ 10117.],
       ...,
       [311217.],
       [311217.],
       [311217.]]), array([[ 0.],
       [ 0.],
       [ 0.],
       ...,
       [23.],
       [23.],
       [23.]]), array([[ 1.],
       [ 1.],
       [ 1.],
       ...,
       [59.],
       [59.],
       [59.